In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\zeyne\\Containerized-Bank-Customer-Churn-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\zeyne\\Containerized-Bank-Customer-Churn-Prediction'

yukarıdaki 4 hücrede dosya işlemleri özellikle dosyaları oluşturma ve aralarında dolanma amaçlı 'OS' kütüphanesini import edip tam olarak hangi konumda olduğumuzu öğreniyoruz

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file :str
    unzip_dir : Path


yukarıdaki hücrede ise config.yaml file'da olan ve sonrasında sıkça kullanacağımız için bir config sınıfına sabitleyip kullanmanın daha avantajlı olacağı 

root_dir --> ilgili işlemin yapıldığı kök dosya

source_URL --> ilgili işlem data almak olduğu için alınacak datanın bağlantısı

local_data_file --> data indiği zaman açılan dosya 

unzip_dir --> datanın boyut kaynaklı zip olması halinde kullanılacak ve oluşturulacak dosya

In [ ]:
from src.Containerized_Bank_Customed_Churn_Prediction.constants import *
from src.Containerized_Bank_Customed_Churn_Prediction.utils.common import read_yaml,create_directories

yukarıdaki hücrede ise kodun yapısını oluştururken constants içine yaml dosyalarının relative path hallerini koymuştuk onları ve common adlı sık kullanılan ve kullanılacak olan fonksiyonlar bulunan filedan yaml dosyalarını okuması için + klasörler oluşturmak için fonksiyonlar import ederiz

In [ ]:
class ConfigurationManager:
 def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH,schema_filepath = SCHEMA_FILE_PATH):

    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    self.schema = read_yaml(schema_filepath)

    create_directories([self.config.artifacts_root])

 def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])
       
       
        data_ingestion_config = DataIngestionConfig(
         root_dir = config.root_dir,
         source_URL = config.source_URL,
         local_data_file = config.local_data_file,
         unzip_dir = config.unzip_dir,
        )
        return data_ingestion_config


yukarıdaki hücrede ilk kısımda ConfigurationManager classı adı üzerinde Config işlemlerini düzenlemek için init fonksiyonunda yaml dosyalarını paramaetre olarak veririz sonrasında python OOP de alışkın olduğumuz self ile onları eşleştiririz

ikinci kısımda ise ilgili işlemle ileride yapılacak işlemlerin config üzerinde olması için class içinde düzenleme yapıyoruz fonksiyonun return ifadesinde DataIngestionConfig nesnesinin dönmesini istiyorum bunun içinde hem fonksiyonun başında hemde içinde config ile config.yaml dosyasının içinde de yukarıdaki hücrelerde de bulunan dosya yolları vs eşitliyoruz 

In [36]:
import os
import urllib.request as request
import zipfile
from Containerized_Bank_Customed_Churn_Prediction import logger
from Containerized_Bank_Customed_Churn_Prediction.utils.common import get_size

urllib kütüphanesinde request modülünü kullanıyoruz amacı basitçe verilen url'den dosyayı alıp lokal bir dosya kurmaktır

In [50]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url= self.config.source_URL,
                filename = self.config.local_data_file

            )
            logger.info(f" {filename} adlı dosya indiriliyor, {headers}")
        else :
            logger.info(f" {filename} adlı dosya zaten var")


    def extract_file(self):
        unzip_path = self.config.unzip_dir,
        os.makedirs(unzip_path,exists_ok=True)
        with zipfile.Zipfile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


yukarıdaki hücrede ilgili işlemin -veri alma- yapabileceği işlemleri barındıran ve onlara ayrılan fonksiyonlar bulunur. classın init fonksiyonunda yukarıdaki hücrelerde hazırlanan config nesnesini kullanıma alıyoruz

download_file fonksiyonunda eğer dosya yoksa config.yamlda bulunan url'i alıp lokal bir data oluşturmasını aksi takdirde dosyanın zaten mevcut olduğuna dair uyarı yapmasını sağlıyoruz

extract_file'da ise datanın zip file olması durumunda 'ayıklayıp' onu lokal başka dosya oluşturup oraya aktarmasını istiyoruz

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_file

except Exception as e:
    raise e

yukarıda ise pipeline kurararak şimdiye kadar hazırladığımız fakat aksiyon almadığımız veri alma işlemini gerçekleştiriyoruz

1) ConfigurationManager sınıfından config nesnesini alıyoruz

2) data_ingestion işlemi için ConfigurationManager sınıfında bulunan get... fonksiyonundan işlem spesifik config alıyoruz

3) Sonrasında data ingestion değişkenini DataIngestion nesnesi olarak görüyoruz, sınıfda parametre olarak verdiğimiz config kısmına hemen bir satır yukarasında eşleştirme yaptığımız configi atıyoruz

4) DataIngestion classının içinde bulunan fonksiyonları çağırıyoruz